In [1]:
DIFF_LIMIT = 36

In [2]:
import os
import pandas as pd
import numpy as np
from numba import njit
import pickle
from itertools import product

In [3]:
data = pd.read_csv("../input/santa-workshop-tour-2019/family_data.csv")


In [4]:
data.head()

,family_id,choice_0,choice_1,choice_2,choice_3,choice_4,choice_5,choice_6,choice_7,choice_8,choice_9,n_people
0,0,52,38,12,82,33,75,64,76,10,28,4
1,1,26,4,82,5,11,47,38,6,66,61,4
2,2,100,54,25,12,27,82,10,89,80,33,3
3,3,2,95,1,96,32,6,40,31,9,59,2
4,4,53,1,47,93,26,3,46,16,42,39,4


In [5]:
def get_penalty(n, choice):
    penalty = None
    if choice == 0:
        penalty = 0
    elif choice == 1:
        penalty = 50
    elif choice == 2:
        penalty = 50 + 9 * n
    elif choice == 3:
        penalty = 100 + 9 * n
    elif choice == 4:
        penalty = 200 + 9 * n
    elif choice == 5:
        penalty = 200 + 18 * n
    elif choice == 6:
        penalty = 300 + 18 * n
    elif choice == 7:
        penalty = 300 + 36 * n
    elif choice == 8:
        penalty = 400 + 36 * n
    elif choice == 9:
        penalty = 500 + 36 * n + 199 * n
    else:
        penalty = 500 + 36 * n + 398 * n
    return penalty


def GetPreferenceCostMatrix(data):
    cost_matrix = np.zeros((N_FAMILIES, N_DAYS), dtype=np.int64)
    for i in range(N_FAMILIES):
        desired = data.values[i, 1:-1]
        cost_matrix[i, :] = get_penalty(FAMILY_SIZE[i], 10)
        for j, day in enumerate(desired):
            cost_matrix[i, day-1] = get_penalty(FAMILY_SIZE[i], j)
    return cost_matrix


def GetAccountingCostMatrix():
    ac = np.zeros((1000, 1000), dtype=np.float64)
    for n in range(ac.shape[0]):
        for n_p1 in range(ac.shape[1]):
            diff = abs(n - n_p1)
            ac[n, n_p1] = max(0, (n - 125) / 400 * n**(0.5 + diff / 50.0))
    return ac
@njit(fastmath=True)
def pcost(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = 0
    for (i, p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty += PCOSTM[i, p]
        daily_occupancy[p] += n
    return penalty, daily_occupancy


# accounting cost
@njit(fastmath=True)
def acost(daily_occupancy):
    accounting_cost = 0
    n_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day + 1]
        n    = daily_occupancy[day]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        accounting_cost += ACOSTM[n, n_p1]
    return accounting_cost, n_out_of_range


@njit(fastmath=True)
def acostd(daily_occupancy):
    accounting_cost = np.zeros(N_DAYS, dtype=np.float64)
    n_out_of_range = 0
    daily_occupancy[-1] = daily_occupancy[-2]
    for day in range(N_DAYS):
        n_p1 = daily_occupancy[day + 1]
        n    = daily_occupancy[day]
        n_out_of_range += (n > MAX_OCCUPANCY) or (n < MIN_OCCUPANCY)
        accounting_cost[day] = ACOSTM[n, n_p1]
    return accounting_cost, n_out_of_range


@njit(fastmath=True)
def pcostd(prediction):
    daily_occupancy = np.zeros(N_DAYS+1, dtype=np.int64)
    penalty = np.empty_like(prediction)
    for (i, p) in enumerate(prediction):
        n = FAMILY_SIZE[i]
        penalty[i] = PCOSTM[i, p]
        daily_occupancy[p] += n
    return penalty, daily_occupancy


@njit(fastmath=True)
def cost_stats(prediction):
    penalty, daily_occupancy = pcostd(prediction)
    accounting_cost, n_out_of_range = acostd(daily_occupancy)
    return penalty, accounting_cost, n_out_of_range, daily_occupancy[:-1]


@njit(fastmath=True)
def cost_function(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, n_out_of_range = acost(daily_occupancy)
    return penalty + accounting_cost + n_out_of_range*100000000

In [6]:
@njit(fastmath=True)
def cost_function_(prediction):
    penalty, daily_occupancy = pcost(prediction)
    accounting_cost, n_out_of_range = acost(daily_occupancy)
    return penalty + accounting_cost, n_out_of_range


@njit(fastmath=True)
def findAnotherDay4Fam(prediction, fam, occupancy):
    old_day = prediction[fam]
    best_cost = np.inf
    best_day = fam
    n = FAMILY_SIZE[fam]
    
    daysrange = list(range(0,old_day))+list(range(old_day+1,N_DAYS))
    for day in daysrange:
        prediction[fam] = day
        new_cost, _ = cost_function_(prediction)
        
        if (new_cost<best_cost) and (occupancy[day]+n<=MAX_OCCUPANCY):
            best_cost = new_cost
            best_day = day
            
    prediction[fam] = old_day
    return best_day, best_cost


@njit(fastmath=True)
def bestFamAdd(prediction, day, occupancy):
    best_cost = np.inf
    best_fam = prediction[day]
    for fam in np.where(prediction!=day)[0]:
        old_day = prediction[fam]
        prediction[fam] = day
        new_cost, _ = cost_function_(prediction)
        prediction[fam] = old_day
        n = FAMILY_SIZE[fam]
        if (new_cost<best_cost) and (occupancy[old_day]-n>=MIN_OCCUPANCY):
            best_cost = new_cost
            best_fam = fam   
    return best_fam


@njit(fastmath=True)
def bestFamRemoval(prediction, day, occupancy):
    best_cost = np.inf
    best_day = day
    
    for fam in np.where(prediction==day)[0]:
        new_day, new_cost = findAnotherDay4Fam(prediction, fam, occupancy)
        if new_cost<best_cost:
            best_cost = new_cost
            best_fam = fam
            best_day = new_day
            
    return best_fam, best_day


@njit(fastmath=True)
def fixMaxOccupancy(prediction):
    penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)

    for day in np.where(occupancy>MAX_OCCUPANCY)[0]:
        while occupancy[day]>MAX_OCCUPANCY:
            fam, new_day = bestFamRemoval(prediction, day, occupancy)
            prediction[fam] = new_day
            penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)
            
            
@njit(fastmath=True)
def fixMinOccupancy(prediction):
    penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)

    for day in np.where(occupancy<MIN_OCCUPANCY)[0]:
        while occupancy[day]<MIN_OCCUPANCY:
            fam = bestFamAdd(prediction, day, occupancy)
            prediction[fam] = day
            penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)

In [7]:
# swappers

def findBetterDay4Family(pred):
    fobs = np.argsort(FAMILY_SIZE)
    score = cost_function(pred)
    original_score = np.inf
    while original_score>score:
        original_score = score
        for family_id in fobs:
            for pick in range(10):
                day = DESIRED[family_id, pick]
                oldvalue = pred[family_id]
                pred[family_id] = day
                new_score = cost_function(pred)
                if new_score<score:
                    score = new_score
                else:
                    pred[family_id] = oldvalue

        print(score, end='\r')
    print(score)
    

def stochastic_product_search(top_k, fam_size, original, 
                              verbose=1000, verbose2=50000,
                              n_iter=500, random_state=2019):
    """
    original (np.array): The original day assignments.
    
    At every iterations, randomly sample fam_size families. Then, given their top_k
    choices, compute the Cartesian product of the families' choices, and compute the
    score for each of those top_k^fam_size products.
    """
    
    best = original.copy()
    best_score = cost_function(best)
    
    np.random.seed(random_state)

    for i in range(n_iter):
        fam_indices = np.random.choice(range(DESIRED.shape[0]), size=fam_size)
        changes = np.array(list(product(*DESIRED[fam_indices, :top_k].tolist())))

        for change in changes:
            new = best.copy()
            new[fam_indices] = change

            new_score = cost_function(new)

            if new_score < best_score:
                best_score = new_score
                best = new
                
        if verbose and i % verbose == 0:
            print(f"Iteration #{i}: Best score is {best_score:.2f}      ", end='\r')
            
        if verbose2 and i % verbose2 == 0:
            print(f"Iteration #{i}: Best score is {best_score:.2f}      ")
    
    print(f"Final best score is {best_score:.2f}")
    return best

# Solver

In [8]:
N_DAYS = 100
N_FAMILIES = 5000
MAX_OCCUPANCY = 300
MIN_OCCUPANCY = 125
FAMILY_SIZE = data.n_people.values
DESIRED     = data.values[:,1:-1] - 1
PCOSTM = GetPreferenceCostMatrix(data)
ACOSTM = GetAccountingCostMatrix()

In [9]:
%%time
from ortools.linear_solver import pywraplp
# BOP_INTEGER_PROGRAMMING - Not Good
# CBC_MIXED_INTEGER_PROGRAMMING -Done
# CLP_LINEAR_PROGRAMMING  - Not constraint
# GLOP_LINEAR_PROGRAMMING - Not constraint
solver = pywraplp.Solver('SolveAssignmentProblemMIP',pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

num_workers = len(PCOSTM)
num_tasks = len(PCOSTM[0])

x = {}
for i in range(num_workers):
    for j in range(num_tasks):
        x[i, j] = solver.BoolVar('x[%i,%i]' % (i, j))

for j in range(num_tasks-1):
    daily_o1 = solver.Sum([x[i, j]*FAMILY_SIZE[i] for i in range(num_workers)])
    daily_o2 = solver.Sum([x[i, j+1]*FAMILY_SIZE[i] for i in range(num_workers)])
    solver.Add((daily_o1-daily_o2)<=DIFF_LIMIT)
    solver.Add((daily_o2-daily_o1)<=DIFF_LIMIT)
        
# every one take one day
for i in range(num_workers):
    solver.Add(solver.Sum([x[i, j] for j in range(num_tasks)]) == 1)

# every day can be taken less than 300 or more than 125
for j in range(num_tasks):
    daily_o = solver.Sum([x[i, j]*FAMILY_SIZE[i] for i in range(num_workers)])
    solver.Add(daily_o<=300)
    solver.Add(daily_o>=125)

CPU times: user 56 s, sys: 400 ms, total: 56.4 s
Wall time: 56.6 s


In [10]:
%%time
# solver.set_time_limit(1000*400)
solver.Minimize(solver.Sum([PCOSTM[i][j] * x[i,j] for i in range(num_workers) for j in range(num_tasks)]))
sol = solver.Solve()
print(sol)

0
CPU times: user 3min 55s, sys: 1.6 s, total: 3min 57s
Wall time: 3min 55s


In [11]:
print(f'Total cost = {solver.Objective().Value():,.0f}')
print()
results = []
for i in range(num_workers):
    for j in range(num_tasks):
        if x[i, j].solution_value() > 0:
#             print('Worker %d assigned to task %d.  Cost = %d' % (
#                   i,
#                   j,
#                   cost_matrix[i][j]))
            results.append((i,j,PCOSTM[i,j]))
         

df = pd.DataFrame(results, columns=['family_id', 'day', 'cost'])
    
if len(df)!=N_FAMILIES:
    df = df.sort_values(['family_id', 'cost']).drop_duplicates('family_id', keep='last') 

Total cost = 67,513



In [12]:
prediction = df.day.values
penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)
print(penalty.sum(), accounting_cost.sum(), n_out_of_range, occupancy.min(), occupancy.max())

70431 11471.216106798205 1 125 304


In [13]:
fixMinOccupancy(prediction)
fixMaxOccupancy(prediction)

penalty, accounting_cost, n_out_of_range, occupancy = cost_stats(prediction)
print(penalty.sum(), accounting_cost.sum(), n_out_of_range, occupancy.min(), occupancy.max())

70594 10404.100156158976 0 125 300


In [14]:
# findBetterDay4Family(data.iloc[:,1].values)

In [15]:
findBetterDay4Family(prediction)

73495.57597740216


In [16]:
final = stochastic_product_search(
        top_k=2,
        fam_size=8, 
        original=prediction, 
        n_iter=350000,
        verbose=1000,
        verbose2=50000 
        )

Iteration #0: Best score is 73495.58      
Iteration #50000: Best score is 73334.16      
Iteration #100000: Best score is 73230.81      
Iteration #150000: Best score is 72978.67      
Iteration #200000: Best score is 72556.49      
Iteration #250000: Best score is 72475.48      
Iteration #300000: Best score is 72344.49      
Final best score is 72333.24


In [17]:
sub = pd.DataFrame(final+1).rename(columns={0:'assigned_day'})
sub['family_id'] = range(N_FAMILIES)
sub[['family_id', 'assigned_day']].to_csv('submission.csv', index=False)